In [23]:
# paf to mapping statistic 
# this program don't support 'mode' and 'gmean' computing
# turn to paf_statistic(feather).ipynb if you need it 

%run data_path.ipynb
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.max_columns = None

start_time = time.time()

singleinput = [umap_files["BBv2"],]

# statn1 = ('read_len','align_len','match')
# statn2 = ('mean','std','25%','50%','75%', 'min', 'max')
statn1 = ['read_len',]
statn2 = ['mean','std','50%']
statn3 = ['mapq','identity', 'read_cover']

refset=set()

def paf2df(ifile):
    df_map={"ref_name":[], "read_len":[], "align_len":[], "match":[], "mapq":[], "identity":[], "read_cover":[]}
    with open(ifile,'r') as f:
        for line in f:
            line = line.split()
            ref = line[5].split('-')
            ref = ref[1].split('_')[0]
            refset.add(ref)
            rlen = int(line[1])
            alilen = int(line[10])
            match = int(line[9])
            mapq = int(line[11])
            df_map["ref_name"].append(ref)
            df_map["read_len"].append(rlen)
            df_map["align_len"].append(alilen)
            df_map["match"].append(match)
            df_map["mapq"].append(mapq)
            df_map["identity"].append(match/alilen)
            df_map["read_cover"].append(alilen/rlen)        
    return pd.DataFrame(df_map)
#     display(paf_df)
#     paf_df.to_feather("zymo_feather/trim_BBv1.feather")
#     paf_df = pd.read_feather("uniq_map.feather")

def build_statdf(df):
    total_record = df.describe()
    stat_list1 = total_record.loc[statn2,statn1].T.values.tolist()
    stat_list1 = [e for i in stat_list1 for e in i]
    stat_list2 = total_record.loc[['mean'],statn3].values.tolist()[0]
    totallennum = total_record.at['count','read_len']

    total_stat = [['Total'] + [totallennum] + [100] + stat_list1 + stat_list2]
    multindex=[('species','species'),('len_number','len_number'),('ratio','ratio')]
    for i in statn1:
        for j in statn2:
            multindex.append((i,j))
    statn3_index = tuple((i,i) for i in statn3)
    multindex += statn3_index

    multindex = pd.Index(multindex)
    return pd.DataFrame(total_stat,columns=multindex)

def display_stat_table(paf_df,stat_df):
    dfdict={}
    for ref in refset:
        dfdict[ref]=paf_df.loc[:][paf_df.ref_name == ref]
        ref_record = dfdict[ref].describe()
        stat_list1 = ref_record.loc[statn2,statn1].T.values.tolist()
        stat_list1 = [e for i in stat_list1 for e in i]
        stat_list2 = ref_record.loc[['mean'],['mapq','identity','read_cover']].values.tolist()[0]
        lennum = ref_record.at['count','read_len']
        ratio = ref_record.at['count','read_len']/int(stat_df.iloc[0]['len_number'])*100
        ref_stat = [ref] + [lennum] + [ratio] + stat_list1 + stat_list2
        stat_df.loc[len(stat_df)] = ref_stat
        stat_df.sort_values(('len_number','len_number'),ascending=False ,inplace=True)
    display(stat_df.round(2))

sdict={}
for inputpath in singleinput:
    print(inputpath)
    sample = inputpath.split('/')[-1]
    sdict[sample] = paf2df(inputpath)
    stat_df = build_statdf(sdict[sample])
    display_stat_table(sdict[sample],stat_df)

    
print("--- %s seconds ---" % (time.time() - start_time))

/big6_disk/fangren106/mock_community/v2_zymo/even/alignment/True/uniq_map.paf
(('mapq', 'mapq'), ('identity', 'identity'), ('read_cover', 'read_cover'))


species len_number   ratio read_len                    mapq  \
           species len_number   ratio     mean      std     50%   mapq   
0            Total  2944724.0  100.00  4001.56  3127.88  3313.0  59.39   
8         Bacillus   604699.0   20.53  3608.36  1968.60  3311.0  59.45   
9    Lactobacillus   481847.0   16.36  2580.50  1979.99  2055.0  59.05   
2         Listeria   436595.0   14.83  3712.71  2089.54  3375.0  59.49   
1     Enterococcus   373286.0   12.68  3732.65  2350.59  3304.0  59.49   
4   Staphylococcus   370654.0   12.59  3688.41  2371.46  3241.0  59.49   
6      Escherichia   200743.0    6.82  6293.36  4848.70  4895.0  59.49   
7       Salmonella   195060.0    6.62  6139.73  4928.83  4755.0  59.45   
10     Pseudomonas   173928.0    5.91  6606.64  4995.04  5238.0  59.40   
3    Saccharomyces    57867.0    1.97  3212.68  1839.86  2903.0  59.12   
5     Cryptococcus    50045.0    1.70  3611.63  1971.44  3280.0  59.28   

   identity read_cover  
   identity read_cover  
0      0.87       1.02  
8      0.87       1.03  
9      0.87       1.02  
2      0.87       1.03  
1      0.87       1.03  
4      0.88       1.02  
6      0.87       1.03  
7      0.87       1.03  
10     0.86       1.03  
3      0.87       1.01  
5      0.87       1.02

--- 46.54115962982178 seconds ---
